In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as U
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import random

### Define the Neural Network

We want to be able to set the weight of the network manually

In [2]:
class NeuralNet(nn.Module):
    def __init__(self, in_dim=2, out_dim=2):
        super(NeuralNet, self).__init__()

        self.layer1=nn.Linear(in_features=in_dim, out_features=4)
        self.layer2=nn.Linear(in_features=4, out_features=out_dim)

        self.weights_initialization()
    
    def weights_initialization(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)  # This is the default in PyTorch
                nn.init.constant_(module.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = F.relu(out)
        out = self.layer2(out)
        return out
    
    def get_flat_params(self):
        return U.parameters_to_vector(self.parameters())
    
    def set_flat_params(self, flat_params):
        U.vector_to_parameters(flat_params, self.parameters())

### Load Data

In [3]:
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

### Evolutionary Algorithm

In our case the flat parameters will be the chromosomes

In [4]:
# Initialize model and get flat parameters
model = NeuralNet()
params_vector = model.get_flat_params()
print(params_vector)


# Configuration
POPULATION_SIZE = 100
CHROMOSOME_LENGTH = len(params_vector)


tensor([ 5.4157e-01, -9.6938e-01,  6.8159e-01, -3.3370e-01,  3.3292e-01,
         6.7682e-01,  9.4453e-01, -1.3159e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  7.5796e-02, -2.5452e-01,  1.1230e-01,
         9.0249e-01, -1.3721e-04, -2.8852e-01,  7.4950e-01,  8.8189e-01,
         0.0000e+00,  0.0000e+00], grad_fn=<CatBackward0>)


In [15]:
def generate_population(size, chromosome_length):
    return [torch.rand(chromosome_length) for _ in range(size)]

def fitness(chromosome, model):
    model.set_flat_params(chromosome)   # Update model's parameters with the chromosome
    with torch.no_grad():
        outputs = model(X_test)
        pred = outputs.argmax(dim=1)
        accuracy = (pred == y_test).float().mean().item()

    return accuracy

def selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    selection_probs = [f / total_fitness for f in fitness_scores]
    parent1 = population[random.choices(range(len(population)), selection_probs)[0]]
    parent2 = population[random.choices(range(len(population)), selection_probs)[0]]

    return parent1, parent2



In [18]:
population = generate_population(POPULATION_SIZE, CHROMOSOME_LENGTH)
fitness_scores = [fitness(individual, model) for individual in population]

print(f"Population Sample: {population[0]}\n")
print(f"Fitness Score: {fitness_scores[0]}\n")
print(f"Selection Result: {selection(population, fitness_scores)}")

Population Sample: tensor([0.0701, 0.1307, 0.6745, 0.1673, 0.8392, 0.8869, 0.2820, 0.0114, 0.7307,
        0.9055, 0.8693, 0.6173, 0.2847, 0.7852, 0.1835, 0.6364, 0.6955, 0.8614,
        0.6639, 0.0148, 0.9623, 0.4542])

Fitness Score: 0.3700000047683716

Selection Result: (tensor([0.5948, 0.8742, 0.7199, 0.1199, 0.8909, 0.6064, 0.7875, 0.5580, 0.0707,
        0.8762, 0.1548, 0.2320, 0.6264, 0.2064, 0.5214, 0.6253, 0.5612, 0.3735,
        0.0711, 0.8735, 0.9077, 0.8904]), tensor([0.8668, 0.1617, 0.7538, 0.9847, 0.9022, 0.9672, 0.6180, 0.0624, 0.8535,
        0.9994, 0.5139, 0.8295, 0.2727, 0.3632, 0.3892, 0.5743, 0.4257, 0.6673,
        0.6631, 0.2655, 0.8874, 0.1056]))
